In [1]:
from ImagePreprocessing import *
import time

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [3]:
### Path settings
train_fraction = 60
val_fraction = 20
data_address = "../data/" # Change this to load files
train_label_filename = "training_data_with_labels.csv"  # New Training 
val_label_filename = "val_data_with_labels.csv"
val_label_small_filename = "validate_labels_small.csv"

### Load Training Set and Validation Set
training_dataset = SnakeDataSet(filename=os.path.join(data_address,train_label_filename),image_path=os.path.join(data_address,'train_images')) 
val_dataset = SnakeDataSet(filename=os.path.join(data_address,val_label_filename),image_path=os.path.join(data_address,'validate_images')) 
training_size = len(training_dataset)
val_size = len(val_dataset)

# There are 783 classes (0-782)

### Load Set to Loader
train_loader = DataLoader(training_dataset, batch_size=64, sampler = SubsetRandomSampler(range(int(training_size / train_fraction))))
val_loader = DataLoader(val_dataset, batch_size=32, sampler = SubsetRandomSampler(range(int(val_size / val_fraction))))

In [4]:
# Random Seed
torch.manual_seed(2020)
np.random.seed(2020)

# Device
device = torch.device("cuda")
model = BasicNet().to(device)
model = model.float()

# Optimizer
optimizer = optim.Adadelta(model.parameters(), lr=1)

# Set your learning rate scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

# Epochs
epochs = 10

In [6]:
# Training loop
train_loss = []
val_loss = []
for epoch in range(1, epochs+1):
    train(model, device, train_loader, train_fraction, optimizer, epoch, 10)
    train_loss.append(test(model, device, train_loader, train_fraction))
    val_loss.append(test(model, device, val_loader, val_fraction))
    scheduler.step()    # learning rate scheduler


Train Epoch: 1 [0/4072 (0%)]	Loss: 7.627436

Time Elapsed: 3 s.

Train Epoch: 1 [640/4072 (16%)]	Loss: 7.411452

Time Elapsed: 35 s.

Train Epoch: 1 [1280/4072 (31%)]	Loss: 6.293200

Time Elapsed: 34 s.

Train Epoch: 1 [1920/4072 (47%)]	Loss: 6.578409

Time Elapsed: 40 s.

Train Epoch: 1 [2560/4072 (63%)]	Loss: 6.365982

Time Elapsed: 40 s.

Train Epoch: 1 [3200/4072 (79%)]	Loss: 6.391438

Time Elapsed: 37 s.

Train Epoch: 1 [3840/4072 (94%)]	Loss: 6.381324

Time Elapsed: 37 s.


Test set: Average loss: 6.2409, Accuracy: 198/4072 (5%)


Test set: Average loss: 6.3687, Accuracy: 32/692 (5%)

Train Epoch: 2 [0/4072 (0%)]	Loss: 6.177875

Time Elapsed: 4 s.

Train Epoch: 2 [640/4072 (16%)]	Loss: 5.551155

Time Elapsed: 35 s.

Train Epoch: 2 [1280/4072 (31%)]	Loss: 6.149891

Time Elapsed: 36 s.

Train Epoch: 2 [1920/4072 (47%)]	Loss: 5.807298

Time Elapsed: 38 s.

Train Epoch: 2 [2560/4072 (63%)]	Loss: 5.779727

Time Elapsed: 40 s.

Train Epoch: 2 [3200/4072 (79%)]	Loss: 5.610975

Time Elap

In [ ]:
torch.save(model.state_dict(), "cnn_model_epoch15_lr1_batchsize32.pt")
f = open("cnn_model_epoch15_lr1_batchsize32_accuracy.txt","w")
f.write('Training Loss: ' + ' '.join(str(_) for _ in train_loss) + '\n')
f.write('Validation Loss: ' + ' '.join(str(_) for _ in val_loss) + '\n')
f.close()